In [1]:
import pandas as pd
import numpy as np
import feather

from tqdm import tqdm

In [2]:
def evaluation_metric(y_true, y_pred):
    y_pred = y_pred.copy()
    y_pred[y_pred < 1.0] = 1.0
    y_pred = np.round(y_pred)
    scores = (y_pred - y_true) / (y_pred + y_true)
    scores[y_true < 1] = 0.0
    return np.abs(scores).mean()

In [3]:
df_base = feather.read_dataframe('features/base_features_no_outliers.feather')
df_mean_window = feather.read_dataframe('features/window_mean_features_noout.feather')
df_trend_window = feather.read_dataframe('features/window_trend_features_noout.feather')
df_week_trend = feather.read_dataframe('features/weekly_trends_features_noout.feather')
df_last_values = feather.read_dataframe('features/last_values_features.feather')
#df_prophet = feather.read_dataframe('features/prophet_features_proper.feather')
#df_views = feather.read_dataframe('features/view_features.feather')
# df_store_simple = feather.read_dataframe('features/store_simple.feather')
# df_store_series = feather.read_dataframe('features/store_series.feather')

df_entities = feather.read_dataframe('features/entity_embedding.feather')

df_pc = feather.read_dataframe('features/pays_count_unmodified.feather')

In [4]:
df_entities['dow'] = df_base.dow

In [5]:
common = (set(df_base.columns) & set(df_mean_window.columns))

In [6]:
for c in common:
    if not (df_base[c].fillna(-1) == df_mean_window[c].fillna(-1)).all():
        print('uwaga')
    if not (df_base[c].fillna(-1) == df_trend_window[c].fillna(-1)).all():
        print('uwaga')
    if not (df_base[c].fillna(-1) == df_week_trend[c].fillna(-1)).all():
        print('uwaga')
    if not (df_base[c].fillna(-1) == df_last_values[c].fillna(-1)).all():
        print(c, 'uwaga')
#     if not (df_base[c].fillna(-1) == df_prophet[c].fillna(-1)).all():
#        print(c, 'uwaga')
#     if not (df_base[c].fillna(-1) == df_views[c].fillna(-1)).all():
#         print(c, 'uwaga')
#     if not (df_base[c].fillna(-1) == df_store_simple[c].fillna(-1)).all():
#         print(c, 'uwaga')
#     if not (df_base[c].fillna(-1) == df_store_series[c].fillna(-1)).all():
#         print(c, 'uwaga')

    if not (df_base[c].fillna(-1) == df_entities[c].fillna(-1)).all():
        print(c, 'uwaga')

    if not (df_base[c].fillna(-1) == df_pc[c].fillna(-1)).all():
        print(c, 'uwaga')

pays_count uwaga


In [7]:
df_mean_window = df_mean_window[sorted(set(df_mean_window.columns) - common)]
df_trend_window = df_trend_window[sorted(set(df_trend_window.columns) - common)]
df_week_trend = df_week_trend[sorted(set(df_week_trend.columns) - common)]
df_last_values = df_last_values[sorted(set(df_last_values.columns) - common)]
#df_views = df_views[sorted(set(df_views.columns) - common)]
#df_store_simple = df_store_simple[sorted(set(df_store_simple.columns) - common)]
#df_store_series = df_store_series[sorted(set(df_store_series.columns) - common)]

df_entities = df_entities[sorted(set(df_entities.columns) - common)]


#df_prophet = df_prophet[sorted(set(df_prophet.columns) - common)]

In [8]:
#del df_views['views_count']

In [9]:
df_features = pd.concat([
        df_base, 
        df_mean_window, 
        df_trend_window, 
        df_week_trend, 
        df_last_values,
        df_entities,
        #df_views,
        #df_store_simple,
        #df_store_series
        #df_prophet,
    ], axis=1)
df_features['pays_count_unmodified'] = df_pc.pays_count_unmodified
df_features['penultimate_unmodified'] = df_pc.penultimate_unmodified

## Some stuff

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
from sklearn.metrics import mean_squared_error

def rmse(y, y_pred):
    y_pred = y_pred[~np.isnan(y)]
    y = y[~np.isnan(y)]

    y = y[~np.isnan(y_pred)]
    y_pred = y_pred[~np.isnan(y_pred)]

    return np.sqrt(mean_squared_error(y, y_pred))

### Median trend features

In [12]:
trend_groups = [
    ('trend', ['trend_2', 'trend_3', 'trend_4', 'trend_5', 'trend_6', 'trend_12', 'trend_18']),
    ('trend_dow', ['trend_dow_3', 'trend_dow_4', 'trend_dow_5', 'trend_dow_6', 
                   'trend_dow_12', 'trend_dow_18']),
    ('trend_weekend', ['trend_weekend_3', 'trend_weekend_4', 'trend_weekend_5',
                       'trend_weekend_6', 'trend_weekend_12', 'trend_weekend_18']),
#     ('pays_count_category_1_mean_trend', ['pays_count_category_1_mean_trend_2', 'pays_count_category_1_mean_trend_3', 'pays_count_category_1_mean_trend_4', 'pays_count_category_1_mean_trend_5', 'pays_count_category_1_mean_trend_6', 'pays_count_category_1_mean_trend_12', 'pays_count_category_1_mean_trend_18']),
#     ('pays_count_category_1_mean_trend_dow', ['pays_count_category_1_mean_trend_dow_3', 'pays_count_category_1_mean_trend_dow_4', 'pays_count_category_1_mean_trend_dow_5', 'pays_count_category_1_mean_trend_dow_6', 
#                    'pays_count_category_1_mean_trend_dow_12', 'pays_count_category_1_mean_trend_dow_18']),
#     ('pays_count_category_1_mean_trend_weekend', ['pays_count_category_1_mean_trend_weekend_3', 'pays_count_category_1_mean_trend_weekend_4', 'pays_count_category_1_mean_trend_weekend_5',
#                        'pays_count_category_1_mean_trend_weekend_6', 'pays_count_category_1_mean_trend_weekend_12', 'pays_count_category_1_mean_trend_weekend_18']),
#     ('pays_count_category_2_mean_trend', ['pays_count_category_2_mean_trend_2', 'pays_count_category_2_mean_trend_3', 'pays_count_category_2_mean_trend_4', 'pays_count_category_2_mean_trend_5', 'pays_count_category_2_mean_trend_6', 'pays_count_category_2_mean_trend_12', 'pays_count_category_2_mean_trend_18']),
#     ('pays_count_category_2_mean_trend_dow', ['pays_count_category_2_mean_trend_dow_3', 'pays_count_category_2_mean_trend_dow_4', 'pays_count_category_2_mean_trend_dow_5', 'pays_count_category_2_mean_trend_dow_6', 
#                    'pays_count_category_2_mean_trend_dow_12', 'pays_count_category_2_mean_trend_dow_18']),
#     ('pays_count_category_2_mean_trend_weekend', ['pays_count_category_2_mean_trend_weekend_3', 'pays_count_category_2_mean_trend_weekend_4', 'pays_count_category_2_mean_trend_weekend_5',
#                        'pays_count_category_2_mean_trend_weekend_6', 'pays_count_category_2_mean_trend_weekend_12', 'pays_count_category_2_mean_trend_weekend_18']),
#     ('pays_count_category_3_mean_trend', ['pays_count_category_3_mean_trend_2', 'pays_count_category_3_mean_trend_3', 'pays_count_category_3_mean_trend_4', 'pays_count_category_3_mean_trend_5', 'pays_count_category_3_mean_trend_6', 'pays_count_category_3_mean_trend_12', 'pays_count_category_3_mean_trend_18']),
#     ('pays_count_category_3_mean_trend_dow', ['pays_count_category_3_mean_trend_dow_3', 'pays_count_category_3_mean_trend_dow_4', 'pays_count_category_3_mean_trend_dow_5', 'pays_count_category_3_mean_trend_dow_6', 
#                    'pays_count_category_3_mean_trend_dow_12', 'pays_count_category_3_mean_trend_dow_18']),
#     ('pays_count_category_3_mean_trend_weekend', ['pays_count_category_3_mean_trend_weekend_3', 'pays_count_category_3_mean_trend_weekend_4', 'pays_count_category_3_mean_trend_weekend_5',
#                        'pays_count_category_3_mean_trend_weekend_6', 'pays_count_category_3_mean_trend_weekend_12', 'pays_count_category_3_mean_trend_weekend_18']),
#     ('views_trend', ['views_trend_2', 'views_trend_3', 'views_trend_4', 'views_trend_5', 
#                      'views_trend_6', 'views_trend_12', 'views_trend_18']),
#     ('views_trend_dow', ['views_trend_dow_3', 'views_trend_dow_4', 'views_trend_dow_5', 
#                          'views_trend_dow_6', 'views_trend_dow_12', 'views_trend_dow_18']),
#     ('views_trend_weekend', ['views_trend_weekend_3', 'views_trend_weekend_4', 'views_trend_weekend_5',
#                        'views_trend_weekend_6', 'views_trend_weekend_12', 'views_trend_weekend_18']),
]

In [13]:
for n, g in trend_groups:
    df_features[n + '_median'] = df_features[g].median(axis=1)
    df_features[n + '_std'] = df_features[g].std(axis=1)

## Model

In [14]:
import xgboost as xgb

In [15]:
to_exclude = {
    #'shop_id',
    'day', 'days_from_beginning', 'pays_count', 'week_id', 'biweek_id',
    'pays_count_unmodified', 'penultimate_unmodified'
}

useless = {
    'mean_12', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'mean_6',
    'out_of_operation', 'penultimate_null',
    'std_12', 'std_2', 'std_3', 'std_4', 'std_5', 'std_6',
    'weekend_mean_12', 'weekend_mean_2', 'weekend_mean_3',
    'weekend_mean_4', 'weekend_mean_5', 'weekend_mean_6',
    'weekend_std_12', 'weekend_std_2', 'weekend_std_3',
    'weekend_std_4', 'weekend_std_5', 'weekend_std_6',
    'views_mean_2', 'views_mean_3', 'views_penultimate_null',
    'views_std_2', 'views_std_3', 'views_weekend_mean_2', 
    'views_weekend_mean_3', 'views_weekend_std_2', 'views_weekend_std_3',
    'cat1_id',
 'is_weekend',
 'pays_count_category_1_mean_mean_2',
 'pays_count_category_1_mean_mean_3',
 'pays_count_category_1_mean_penultimate_null',
 'pays_count_category_1_mean_std_2',
 'pays_count_category_1_mean_std_3',
 'pays_count_category_1_mean_weekend_mean_2',
 'pays_count_category_1_mean_weekend_mean_3',
 'pays_count_category_1_mean_weekend_std_2',
 'pays_count_category_1_mean_weekend_std_3',
 'pays_count_category_2_mean_mean_2',
 'pays_count_category_2_mean_mean_3',
 'pays_count_category_2_mean_penultimate_null',
 'pays_count_category_2_mean_second_in_biweek',
 'pays_count_category_2_mean_std_2',
 'pays_count_category_2_mean_std_3',
 'pays_count_category_2_mean_weekend_mean_2',
 'pays_count_category_2_mean_weekend_mean_3',
 'pays_count_category_2_mean_weekend_std_2',
 'pays_count_category_2_mean_weekend_std_3',
 'pays_count_category_3_mean_mean_2',
 'pays_count_category_3_mean_mean_3',
 'pays_count_category_3_mean_penultimate_null',
 'pays_count_category_3_mean_second_in_biweek',
 'pays_count_category_3_mean_std_2',
 'pays_count_category_3_mean_std_3',
 'pays_count_category_3_mean_weekend_mean_2',
 'pays_count_category_3_mean_weekend_mean_3',
 'pays_count_category_3_mean_weekend_std_2',
 'pays_count_category_3_mean_weekend_std_3',
 'pays_count_city_name_mean_mean_2',
 'pays_count_city_name_mean_mean_3',
 'pays_count_city_name_mean_penultimate_null',
 'pays_count_city_name_mean_second_in_biweek',
 'pays_count_city_name_mean_std_2',
 'pays_count_city_name_mean_std_3',
 'pays_count_city_name_mean_weekend_mean_2',
 'pays_count_city_name_mean_weekend_mean_3',
 'pays_count_city_name_mean_weekend_std_2',
 'pays_count_city_name_mean_weekend_std_3',
 'pays_count_location_id_mean_mean_2',
 'pays_count_location_id_mean_mean_3',
 'pays_count_location_id_mean_penultimate_null',
 'pays_count_location_id_mean_second_in_biweek',
 'pays_count_location_id_mean_std_2',
 'pays_count_location_id_mean_std_3',
 'pays_count_location_id_mean_weekend_mean_2',
 'pays_count_location_id_mean_weekend_mean_3',
 'pays_count_location_id_mean_weekend_std_2',
 'pays_count_location_id_mean_weekend_std_3',
}

features = sorted(set(df_features.columns) - to_exclude - useless)
print(features)

['biweekly_coef', 'biweekly_dow_coef', 'biweekly_dow_trend', 'biweekly_trend', 'biweekly_weekend_coef', 'biweekly_weekend_trend', 'category_1_0', 'category_1_1', 'category_2_0', 'category_2_1', 'category_2_2', 'category_3_0', 'category_3_1', 'category_3_2', 'category_3_3', 'city_name_0', 'city_name_1', 'city_name_2', 'city_name_3', 'city_name_4', 'city_name_5', 'city_name_6', 'city_name_7', 'dow', 'dow_0', 'dow_1', 'dow_2', 'dow_3', 'dow_4', 'dow_5', 'dow_mean_12', 'dow_mean_2', 'dow_mean_3', 'dow_mean_4', 'dow_mean_5', 'dow_mean_6', 'dow_std_12', 'dow_std_2', 'dow_std_3', 'dow_std_4', 'dow_std_5', 'dow_std_6', 'location_id_0', 'location_id_1', 'location_id_2', 'location_id_3', 'location_id_4', 'location_id_5', 'location_id_6', 'location_id_7', 'location_id_8', 'location_id_9', 'mean_1', 'ooo_mean_1', 'ooo_mean_2', 'ooo_mean_3', 'ooo_mean_4', 'pays_two_weeks_ago', 'penultimate', 'prev_biweek_last_value', 'prev_biweek_p25', 'prev_biweek_p75', 'prev_spread', 'second_in_biweek', 'shop_id'

In [16]:
len(features)

130

In [17]:
def eval_metric(preds, dtrain):
    y_true = dtrain.get_label()
    return 'metric', evaluation_metric(y_true, preds)

In [18]:
xgb_pars = {
    'eta': 0.1,
    'gamma': 0,
    'max_depth': 12,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'lambda': 1,
    'alpha': 0,
    'tree_method': 'approx',
    'objective': 'reg:linear',
    #'objective': 'count:poisson',
    'eval_metric': 'rmse',
    'nthread': 8,
    'seed': 42,
    'silent': 1
}

n_estimators = 70

In [19]:
df_features['pred'] = np.nan

evals = []
rmses = []

for i in tqdm([1, 2, 3, 4]):
    df_train = df_features[df_features.biweek_id > i].reset_index(drop=1)
    df_val = df_features[df_features.biweek_id == i].reset_index(drop=1)

    df_train = df_train[~df_train.pays_count.isnull()].reset_index(drop=1)
    df_val = df_val[~df_val.pays_count_unmodified.isnull()].reset_index(drop=1)

    y_train = df_train.pays_count.values
    y_val = df_val.pays_count_unmodified.values

    X_train = df_train[features].values
    X_val = df_val[features].values

    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features, missing=np.nan)
    dval = xgb.DMatrix(X_val, label=y_val, feature_names=features, missing=np.nan)
    model = xgb.train(xgb_pars, dtrain, num_boost_round=n_estimators)

    y_pred = model.predict(dval)
    val_idx = (df_features.biweek_id == i) & (~df_features.pays_count_unmodified.isnull())
    df_features.loc[val_idx, 'pred'] = y_pred

    r = rmse(y_val, y_pred)
    e = evaluation_metric(y_val, y_pred)

    evals.append(e)
    rmses.append(r)

100%|██████████| 4/4 [08:09<00:00, 122.36s/it]


In [20]:
print(np.mean(rmses), np.std(rmses))
print(np.mean(evals), np.std(evals))

59.1484369734 4.11668717175
0.11697823959 0.0133226101113


In [21]:
from sklearn.ensemble import ExtraTreesRegressor

In [22]:
et_params = dict(
    n_estimators=100,
    criterion='mse',
    max_depth=30,
    min_samples_split=6,
    min_samples_leaf=6,
    max_features=4, 
    bootstrap=False, 
    n_jobs=-1,
    random_state=1
)

In [23]:
df_features['pred_et'] = np.nan

evals = []
rmses = []

for i in tqdm([1, 2, 3, 4]):
    df_train = df_features[df_features.biweek_id > i].reset_index(drop=1)
    df_val = df_features[df_features.biweek_id == i].reset_index(drop=1)

    df_train = df_train[~df_train.pays_count.isnull()].reset_index(drop=1)
    df_val = df_val[~df_val.pays_count_unmodified.isnull()].reset_index(drop=1)

    y_train = df_train.pays_count.values
    y_val = df_val.pays_count_unmodified.values

    X_train = df_train[features].fillna(-1).values
    X_val = df_val[features].fillna(-1).values

    et = ExtraTreesRegressor(**et_params)
    et.fit(X_train, y_train)
    y_pred = et.predict(X_val)

    val_idx = (df_features.biweek_id == i) & (~df_features.pays_count_unmodified.isnull())
    df_features.loc[val_idx, 'pred_et'] = y_pred
    
    r = rmse(y_val, y_pred)
    e = evaluation_metric(y_val, y_pred)

    evals.append(e)
    rmses.append(r)

100%|██████████| 4/4 [00:56<00:00, 14.11s/it]


In [24]:
print(np.mean(rmses), np.std(rmses))
print(np.mean(evals), np.std(evals))

56.4404052145 4.17597846704
0.120954603449 0.0108652292588


In [35]:
evals = []
rmses = []

for i in tqdm([1, 2, 3, 4]):
    df_val = df_features[df_features.biweek_id == i].reset_index(drop=1)
    df_val = df_val[~df_val.pays_count_unmodified.isnull()].reset_index(drop=1)
    y_val = df_val.pays_count_unmodified.values

    xgb_pred = df_val.pred.values
    et_pred = df_val.pred_et.values
    y_pred = (3 * xgb_pred + 1 * et_pred) / 4

    r = rmse(y_val, y_pred)
    e = evaluation_metric(y_val, y_pred)

    evals.append(e)
    rmses.append(r)

print(np.mean(rmses), np.std(rmses))
print(np.mean(evals), np.std(evals))

100%|██████████| 4/4 [00:00<00:00, 22.40it/s]

57.3178979979 3.9974898377
0.116969866005 0.0126765164031


In [ ]:
i = 1
df_train = df_features[df_features.biweek_id > i].reset_index(drop=1)
df_val = df_features[df_features.biweek_id == i].reset_index(drop=1)

df_train = df_train[~df_train.pays_count.isnull()].reset_index(drop=1)
df_val = df_val[~df_val.pays_count_unmodified.isnull()].reset_index(drop=1)

y_train = df_train.pays_count.values
y_val = df_val.pays_count_unmodified.values

X_train = df_train[features].values
X_val = df_val[features].values

dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features, missing=np.nan)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features, missing=np.nan)
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [39]:
xgb_pars = {
    'eta': 0.1,
    'gamma': 0,
    'max_depth': 12,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'lambda': 1,
    'alpha': 0,
    'tree_method': 'approx',
    'objective': 'reg:linear',
    #'objective': 'count:poisson',
    'eval_metric': 'rmse',
    'nthread': 8,
    'seed': 42,
    'silent': 1
}

n_estimators = 70

In [ ]:
model = xgb.train(xgb_pars, dtrain, num_boost_round=n_estimators, evals=watchlist, 
                  verbose_eval=1, feval=eval_metric)

[0]	train-rmse:156.263	val-rmse:189.873	train-metric:0.792569	val-metric:0.80063
[1]	train-rmse:141.64	val-rmse:173.156	train-metric:0.652692	val-metric:0.661842
[2]	train-rmse:128.52	val-rmse:158.505	train-metric:0.546564	val-metric:0.556942
[3]	train-rmse:116.769	val-rmse:145.019	train-metric:0.463802	val-metric:0.474416
[4]	train-rmse:106.215	val-rmse:133.289	train-metric:0.397614	val-metric:0.407687
[5]	train-rmse:96.774	val-rmse:122.797	train-metric:0.343971	val-metric:0.3546
[6]	train-rmse:88.3167	val-rmse:113.754	train-metric:0.30015	val-metric:0.310359
[7]	train-rmse:80.768	val-rmse:105.396	train-metric:0.264029	val-metric:0.273557
[8]	train-rmse:74.0407	val-rmse:98.6084	train-metric:0.234225	val-metric:0.243848
[9]	train-rmse:68.0344	val-rmse:92.2743	train-metric:0.209456	val-metric:0.218786
[10]	train-rmse:62.6906	val-rmse:86.9279	train-metric:0.188919	val-metric:0.197756
[11]	train-rmse:57.9127	val-rmse:82.5198	train-metric:0.171853	val-metric:0.181234
[12]	train-rmse:53.682

In [21]:
scores = model.get_score(importance_type='gain')
sorted(scores.items(), key=lambda x: -x[1])

[('trend_weekend_median', 113158448.22088234),
 ('trend_dow_median', 97180391.08636361),
 ('trend_median', 57006583.06192307),
 ('location_id', 26938969.343623374),
 ('trend_weekend_6', 13020910.39246154),
 ('dow_std_12', 5908357.5626754705),
 ('shop_id', 4335415.3214000035),
 ('trend_dow_12', 3286941.020923695),
 ('pays_count_category_2_mean_trend_dow_6', 2561957.192933334),
 ('trend_2', 2165176.7196511626),
 ('pays_count_category_3_mean_trend_weekend_18', 1588700.009302326),
 ('trend_weekend_coef_12', 1572212.5506666668),
 ('trend_dow_18', 1465860.336859156),
 ('no_user_info', 1336832.6),
 ('dow_std_2', 1178398.6259062497),
 ('pays_count_category_3_mean_trend_dow_18', 1147492.54625),
 ('loc_cnt', 1142407.4385357143),
 ('dow_mean_6', 1109656.6095925926),
 ('pays_count_category_1_mean_trend_dow_12', 1098164.747096774),
 ('trend_5', 1069983.8750000002),
 ('pays_count_location_id_mean_trend_coef_12', 1052398.6981818178),
 ('trend_dow_coef_18', 926727.4659322035),
 ('dow_std_3', 922202.72

In [22]:
set(features) - scores.keys()

{'cat1_id',
 'is_weekend',
 'pays_count_category_1_mean_mean_2',
 'pays_count_category_1_mean_mean_3',
 'pays_count_category_1_mean_penultimate_null',
 'pays_count_category_1_mean_std_2',
 'pays_count_category_1_mean_std_3',
 'pays_count_category_1_mean_weekend_mean_2',
 'pays_count_category_1_mean_weekend_mean_3',
 'pays_count_category_1_mean_weekend_std_2',
 'pays_count_category_1_mean_weekend_std_3',
 'pays_count_category_2_mean_mean_2',
 'pays_count_category_2_mean_mean_3',
 'pays_count_category_2_mean_penultimate_null',
 'pays_count_category_2_mean_second_in_biweek',
 'pays_count_category_2_mean_std_2',
 'pays_count_category_2_mean_std_3',
 'pays_count_category_2_mean_weekend_mean_2',
 'pays_count_category_2_mean_weekend_mean_3',
 'pays_count_category_2_mean_weekend_std_2',
 'pays_count_category_2_mean_weekend_std_3',
 'pays_count_category_3_mean_mean_2',
 'pays_count_category_3_mean_mean_3',
 'pays_count_category_3_mean_penultimate_null',
 'pays_count_category_3_mean_second_in_bi

In [28]:
df_train = df_features[df_features.biweek_id > 0].reset_index(drop=1)
df_train = df_train[~df_train.pays_count.isnull()].reset_index(drop=1)

y_train = df_train.pays_count.values
X_train = df_train[features].values

dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features, missing=np.nan)
model = xgb.train(xgb_pars, dtrain, num_boost_round=n_estimators)

Submission

In [29]:
df_test = df_features[df_features.biweek_id == 0].reset_index(drop=1)

In [30]:
X_test = df_test[features].values
dtest = xgb.DMatrix(X_test, feature_names=features, missing=np.nan)

In [31]:
y_pred = model.predict(dtest)
y_pred[y_pred < 1] = 1
y_pred = np.round(y_pred).astype(int)
df_test['pred'] = y_pred

In [32]:
shops = sorted(set(df_test.shop_id))

In [33]:
with open('sub8.csv', 'w') as f:
    for i in tqdm(shops):
        df_shop = df_test[df_test.shop_id == i]
        f.write(str(i) + ',')
        #if i == 810:
        #    f.write(','.join(str(int(d)) for d in df_shop.penultimate_unmodified))
        #else:
        f.write(','.join(str(d) for d in df_shop.pred))
        f.write('\n')

100%|██████████| 2000/2000 [00:01<00:00, 1217.55it/s]


In [35]:
!head sub8.csv

1,224,219,224,253,239,246,203,213,214,213,247,242,249,212
2,83,87,84,95,112,104,82,86,90,88,98,111,106,85
3,71,73,72,81,102,98,72,72,74,73,83,103,97,73
4,100,110,103,112,172,173,93,94,108,94,116,171,168,94
5,277,161,204,198,216,287,288,289,201,200,208,209,286,310
6,98,98,101,105,104,93,93,98,97,100,105,103,89,94
7,119,107,108,126,163,173,112,119,109,111,137,167,174,113
8,52,52,51,48,72,72,50,53,53,54,49,71,71,50
9,219,223,221,224,271,274,221,221,225,217,225,269,275,220
10,100,111,104,104,105,105,106,103,111,103,103,105,105,110
